# MPIA Arxiv on Deck 2: Debugging notebook

In this notebook, I keep some first order commands for diagnostic of issues with papers.
Main definitions are taken from the main notebook.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

# Some useful definitions.
class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

We get the author list from the MPIA website

In [2]:
# Getting the list of authors can take sometimes (internet connection)
# Caching the MPIA author list to avoid running this line every time we restart the kernel.
import yaml
try:
    with open('tmp_mpia_authors.yml', 'r') as fin:
        mpia_authors = yaml.load(fin, yaml.BaseLoader)
    print("`mpia.get_mpia_mitarbeiter_list()`: restored from cache")
except FileNotFoundError:
    print("`mpia.get_mpia_mitarbeiter_list()`: cannot be restored from cache.")
    # get list from MPIA website
    # it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
    mpia_authors = mpia.get_mpia_mitarbeiter_list()
    with open('tmp_mpia_authors.yml', 'w') as fout:
        fout.write(yaml.dump(mpia_authors))

`mpia.get_mpia_mitarbeiter_list()`: restored from cache


In [3]:
# Get family names only
data = [k[-1] for k in mpia_authors]

# clean -- Having titles is something new at MPIA
data = [k.replace('Dr. ', '').strip() for k in data]

filtered_data = list(filter(mpia.filter_non_scientists, data))

name_variations = filter(lambda x: x is not None,
                         [mpia.consider_variations(name) for name in filtered_data])
mitarbeiter_list = sorted(filtered_data + list(name_variations))

lst = [(mpia.get_special_corrections(mpia.get_initials(name)), name) for name in mitarbeiter_list]
lst = [(mpia.family_name_from_initials(k[0]), k[0], k[1]) for k in lst]
lst = sorted(lst, key=lambda x: x[0])
lst = set(lst)

We get the paper to debug

In [4]:
which = "2302.07916"
paper = get_paper_from_identifier(which)
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2302.07916-b31b1b.svg)](https://arxiv.org/abs/2302.07916) | **The CARMENES search for exoplanets around M dwarfs. Line-by-line sensitivity to activity in M dwarfs**  |
|| M. Lafarga, et al. |
|*Appeared on*| *2023-02-15*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Radial velocities (RVs) measured from high-resolution stellar spectra areroutinely used to detect and characterise orbiting exoplanet companions. Thedifferent lines present in stellar spectra are created by several species,which are non-uniformly affected by stellar variability features such as spotsor faculae. Stellar variability distorts the shape of the spectral absorptionlines from which precise RVs are measured, posing one of the main problems inthe study of exoplanets. In this work we aim to study how the spectral linespresent in M dwarfs are independently impacted by stellar activity. We usedCARMENES optical spectra of six active early- and mid-type M dwarfs to computeline-by-line RVs and study their correlation with several well-studied proxiesof stellar activity. We are able to classify spectral lines based on theirsensitivity to activity in five M dwarfs displaying high levels of stellaractivity. We further used this line classification to compute RVs withactivity-sensitive lines and less sensitive lines, enhancing or mitigatingstellar activity effects in the RV time series. For specific sets of the leastactivity-sensitive lines, the RV scatter decreases by ~ 2 to 5 times theinitial one, depending on the star. Finally, we compare these lines in thedifferent stars analysed, finding the sensitivity to activity to vary from starto star. Despite the high density of lines and blends present in M dwarfstellar spectra, we find that a line-by-line approach is able to deliverprecise RVs. Line-by-line RVs are also sensitive to stellar activity effects,and they allow for an accurate selection of activity-insensitive lines tomitigate activity effects in RV. However, we find stellar activity effects tovary in the same insensitive lines from star to star.|

In [5]:
# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in lst]

hl_authors = highlight_authors_in_list(paper['authors'], hl_list, verbose=True)
matches = [(hl, orig) for hl, orig in zip(hl_authors, paper['authors']) if 'mark' in hl]
if not matches:
    warnings.warn(AffiliationWarning("WARNING: This paper does not seem to have MPIA authors."))
paper['authors'] = hl_authors
paper


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2302.07916-b31b1b.svg)](https://arxiv.org/abs/2302.07916) | **The CARMENES search for exoplanets around M dwarfs. Line-by-line sensitivity to activity in M dwarfs**  |
|| M. Lafarga, et al. -- incl., <mark>Th. Henning</mark>, <mark>M. Kürster</mark> |
|*Appeared on*| *2023-02-15*|
|*Comments*| *Accepted for publication in A&A*|
|**Abstract**| Radial velocities (RVs) measured from high-resolution stellar spectra areroutinely used to detect and characterise orbiting exoplanet companions. Thedifferent lines present in stellar spectra are created by several species,which are non-uniformly affected by stellar variability features such as spotsor faculae. Stellar variability distorts the shape of the spectral absorptionlines from which precise RVs are measured, posing one of the main problems inthe study of exoplanets. In this work we aim to study how the spectral linespresent in M dwarfs are independently impacted by stellar activity. We usedCARMENES optical spectra of six active early- and mid-type M dwarfs to computeline-by-line RVs and study their correlation with several well-studied proxiesof stellar activity. We are able to classify spectral lines based on theirsensitivity to activity in five M dwarfs displaying high levels of stellaractivity. We further used this line classification to compute RVs withactivity-sensitive lines and less sensitive lines, enhancing or mitigatingstellar activity effects in the RV time series. For specific sets of the leastactivity-sensitive lines, the RV scatter decreases by ~ 2 to 5 times theinitial one, depending on the star. Finally, we compare these lines in thedifferent stars analysed, finding the sensitivity to activity to vary from starto star. Despite the high density of lines and blends present in M dwarfstellar spectra, we find that a line-by-line approach is able to deliverprecise RVs. Line-by-line RVs are also sensitive to stellar activity effects,and they allow for an accurate selection of activity-insensitive lines tomitigate activity effects in RV. However, we find stellar activity effects tovary in the same insensitive lines from star to star.|

We get the (TeX) source
* retrieve the tarball
* find the main tex file and parse it
* parse for affiliations (but debugging so we do not stop if not found)
* generate the the output markdown

In [8]:
paper_id = f'{which:s}'
folder = f'tmp_{paper_id:s}'

if not os.path.isdir(folder):
    folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')

try:
    doc = latex.LatexDocument(folder, validation=validation)    
except AffiliationError as affilerror:
    msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
    print(msg)

# Hack because sometimes author parsing does not work well
if (len(doc.authors) != len(paper['authors'])):
    doc._authors = paper['authors']
if (doc.abstract) in (None, ''):
    doc._abstract = paper['abstract']

doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
doc.highlight_authors_in_list(hl_list, verbose=True)

full_md = doc.generate_markdown_text()

/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:119: LatexWarning: attempting recovering figure figures/J07446+035/rvmethod_line7096_J07446+035
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:119: LatexWarning: attempting recovering figure figures/J07446+035/stats_rvlineerr_w_2c
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:119: LatexWarning: attempting recovering figure figures/J07446+035/ts_corr_ccfrvnzp_ccffwhm_ccfcontrast_ccfbis_servalcrx_servaldlw_servalhalpha_fitgaussianrvline_l06661.25_l07855.92_ph
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/home/fouesneau/Work/projects/arxiv_display/venv/lib/python3.10/site-packages/arxiv_on_deck_2/latex.py:119: L

In [9]:
print(doc.source)

\documentclass{aa}
\bibpunct{(}{)}{;}{a} {,}
\usepackage{graphicx}
\usepackage{subcaption}
\usepackage{xspace}
\usepackage{booktabs}
\maxdeadcycles=1000
\usepackage{txfonts}
\usepackage[colorlinks=true,citecolor=blue]{hyperref}
\newcommand{çf}{cross-correlation function\xspace}
\newcommand{\vsini}{\ensuremath{v \sin{i}}\xspace}
\newcommand{\ms}{\ensuremath{\mathrm{m s^{-1}}}\xspace}
\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}\xspace}
\newcommand{\A}{\ensuremath{\mathrm{\AA}}\xspace}
\newcommand{\Halpha}{\ensuremath{\mathrm{H\alpha}}\xspace}
\newcommand{\CaHK}{\ion{Ca}{II} H\&K\xspace}
\newcommand{\pEWHalpha}{pEW\ensuremath{'(\Halpha)}\xspace}
\newcommand{\IHalpha}{\ensuremath{I_{\Halpha}}\xspace}
\newcommand{\logLHalphaLbol}{\ensuremath{\log(L_{\Halpha}/L_{\mathrm{bol}})}\xspace}
\newcommand{\LXLJ}{\ensuremath{L_X/L_J}\xspace}
\newcommand{\RHK}{\ensuremath{R'_{\mathrm{HK}}}\xspace}
\newcommand{\logRHK}{\ensuremath{\log R'_{\mathrm{HK}}}\xspace}
\newcommand{\HeD}{\ion{He}{I} \ensu

In [10]:
print(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{çf}{cross-correlation function\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}\xspace}$
$\newcommand{\ms}{\ensuremath{\mathrm{m s^{-1}}}\xspace}$
$\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}\xspace}$
$\newcommand{\A}{\ensuremath{\mathrm{\AA}}\xspace}$
$\newcommand{\Halpha}{\ensuremath{\mathrm{H\alpha}}\xspace}$
$\newcommand{\CaHK}{\ion{Ca}{II} H\&K\xspace}$
$\newcommand{\pEWHalpha}{pEW\ensuremath{'(\Halpha)}\xspace}$
$\newcommand{\IHalpha}{\ensuremath{I_{\Halpha}}\xspace}$
$\newcommand{\logLHalphaLbol}{\ensuremath{\log(L_{\Halpha}/L_{\mathrm{bol}})}\xspace}$
$\newcommand{\LXLJ}{\ensuremath{L_X/L_J}\xspace}$
$\newco

In [11]:
from IPython.display import Markdown
Markdown(full_md)

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{çf}{cross-correlation function\xspace}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}\xspace}$
$\newcommand{\ms}{\ensuremath{\mathrm{m s^{-1}}}\xspace}$
$\newcommand{\kms}{\ensuremath{\mathrm{km s^{-1}}}\xspace}$
$\newcommand{\A}{\ensuremath{\mathrm{\AA}}\xspace}$
$\newcommand{\Halpha}{\ensuremath{\mathrm{H\alpha}}\xspace}$
$\newcommand{\CaHK}{\ion{Ca}{II} H\&K\xspace}$
$\newcommand{\pEWHalpha}{pEW\ensuremath{'(\Halpha)}\xspace}$
$\newcommand{\IHalpha}{\ensuremath{I_{\Halpha}}\xspace}$
$\newcommand{\logLHalphaLbol}{\ensuremath{\log(L_{\Halpha}/L_{\mathrm{bol}})}\xspace}$
$\newcommand{\LXLJ}{\ensuremath{L_X/L_J}\xspace}$
$\newcommand{\RHK}{\ensuremath{R'_{\mathrm{HK}}}\xspace}$
$\newcommand{\logRHK}{\ensuremath{\log R'_{\mathrm{HK}}}\xspace}$
$\newcommand{\HeD}{\ion{He}{I} \ensuremath{\mathrm{D_3}}\xspace}$
$\newcommand{\Heten}{\ensuremath{\mathrm{\ion{He}{I}} \lambda10833 \mathrm{\AA}}\xspace}$
$\newcommand{\Pabeta}{\ensuremath{\mathrm{Pa}\beta}\xspace}$
$\newcommand{\NaD}{\ensuremath{\mathrm{\ion{Na}{I} D}}\xspace}$
$\newcommand{\Caii}{\ensuremath{\mathrm{\ion{Ca}{II}}}\xspace}$
$\newcommand{\Prot}{\ensuremath{P_{\mathrm{rot}}}\xspace}$
$\newcommand{\Prothalf}{\ensuremath{\frac{1}{2}P_{\mathrm{rot}}}\xspace}$
$\newcommand{\Protthird}{\ensuremath{\frac{1}{3}P_{\mathrm{rot}}}\xspace}$
$\newcommand{\Protalias}{\ensuremath{P_{\mathrm{rot}}\mathrm{ 1 d a.}}\xspace}$
$\newcommand{\Prothalfalias}{\ensuremath{\frac{1}{2}P_{\mathrm{rot}}\mathrm{ 1 d a.}}\xspace}$
$\newcommand{\Protthirdalias}{\ensuremath{\frac{1}{3}P_{\mathrm{rot}}\mathrm{ 1 d a.}}\xspace}$
$\newcommand{\days}{\ensuremath{\mathrm{d}}\xspace}$
$\newcommand{\FAP}{\ensuremath{\mathrm{FAP}}\xspace}$
$\newcommand{\Msun}{\ensuremath{\mathrm{M_{\odot}}}\xspace}$
$\newcommand{\Mearth}{\ensuremath{\mathrm{M_{\oplus}}}\xspace}$
$\newcommand{\Mjup}{\ensuremath{\mathrm{M_{J}}}\xspace}$
$\newcommand{\caracal}{\texttt{caracal}\xspace}$
$\newcommand{\serval}{\texttt{serval}\xspace}$
$\newcommand{\raccoon}{\texttt{raccoon}\xspace}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$
$\newcommand{çf}{cross-correlation function$\xspace$}$
$\newcommand{$\vsini$}{$\ensuremath${v \sin{i}}$\xspace$}$
$\newcommand{$\ms$}{$\ensuremath${\mathrm{m s^{-1}}}$\xspace$}$
$\newcommand{$\kms$}{$\ensuremath${\mathrm{km s^{-1}}}$\xspace$}$
$\newcommand{$\A$}{$\ensuremath${\mathrm{$\A$A}}$\xspace$}$
$\newcommand{$\Halpha$}{$\ensuremath${\mathrm{H\alpha}}$\xspace$}$
$\newcommand{$\CaHK$}{$\ion${Ca}{II} H\&K$\xspace$}$
$\newcommand{$\pEWHalpha$}{pEW$\ensuremath${'($\Halpha$)}$\xspace$}$
$\newcommand{$\IHalpha$}{$\ensuremath${I_{$\Halpha$}}$\xspace$}$
$\newcommand{$\logLHalphaLbol$}{$\ensuremath${\log(L_{$\Halpha$}/L_{\mathrm{bol}})}$\xspace$}$
$\newcommand{$\LXLJ$}{$\ensuremath${L_X/L_J}$\xspace$}$
$\newcommand{$\RHK$}{$\ensuremath${R'_{\mathrm{HK}}}$\xspace$}$
$\newcommand{$\logRHK$}{$\ensuremath${\log R'_{\mathrm{HK}}}$\xspace$}$
$\newcommand{$\HeD$}{$\ion${He}{I} $\ensuremath${\mathrm{D_3}}$\xspace$}$
$\newcommand{$\Heten$}{$\ensuremath${\mathrm{$\ion${He}{I}} \lambda10833 \mathrm{$\A$A}}$\xspace$}$
$\newcommand{$\Pabeta$}{$\ensuremath${\mathrm{Pa}\beta}$\xspace$}$
$\newcommand{$\NaD$}{$\ensuremath${\mathrm{$\ion${Na}{I} D}}$\xspace$}$
$\newcommand{$\Caii$}{$\ensuremath${\mathrm{$\ion${Ca}{II}}}$\xspace$}$
$\newcommand{$\Prot$}{$\ensuremath${P_{\mathrm{rot}}}$\xspace$}$
$\newcommand{$\Prot$half}{$\ensuremath${\frac{1}{2}P_{\mathrm{rot}}}$\xspace$}$
$\newcommand{$\Prot$third}{$\ensuremath${\frac{1}{3}P_{\mathrm{rot}}}$\xspace$}$
$\newcommand{$\Prot$alias}{$\ensuremath${P_{\mathrm{rot}}\mathrm{ 1 d a.}}$\xspace$}$
$\newcommand{$\Prot$halfalias}{$\ensuremath${\frac{1}{2}P_{\mathrm{rot}}\mathrm{ 1 d a.}}$\xspace$}$
$\newcommand{$\Prot$thirdalias}{$\ensuremath${\frac{1}{3}P_{\mathrm{rot}}\mathrm{ 1 d a.}}$\xspace$}$
$\newcommand{$\days$}{$\ensuremath${\mathrm{d}}$\xspace$}$
$\newcommand{$\FAP$}{$\ensuremath${\mathrm{FAP}}$\xspace$}$
$\newcommand{$\Msun$}{$\ensuremath${\mathrm{M_{\odot}}}$\xspace$}$
$\newcommand{$\Mearth$}{$\ensuremath${\mathrm{M_{\oplus}}}$\xspace$}$
$\newcommand{$\Mjup$}{$\ensuremath${\mathrm{M_{J}}}$\xspace$}$
$\newcommand{$\caracal$}{\texttt{caracal}$\xspace$}$
$\newcommand{$\serval$}{\texttt{serval}$\xspace$}$
$\newcommand{$\raccoon$}{\texttt{raccoon}$\xspace$}$</div>



<div id="title">

# The CARMENES search for exoplanets around M dwarfs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2302.07916-b31b1b.svg)](https://arxiv.org/abs/2302.07916) _Accepted for publication in A&A_

</div>
<div id="authors">

M. Lafarga, et al. -- incl., <mark>Th. Henning</mark>, <mark>M. Kürster</mark>

</div>
<div id="abstract">

**Abstract:** Radial velocities (RVs) measured from high-resolution stellar spectra are routinely used to detect and characterise orbiting exoplanet companions. The different lines present in stellar spectra are created by several species, which are non-uniformly affected by stellar variability features such as spots or faculae. Stellar variability distorts the shape of the spectral absorption lines from which precise RVs  are measured, posing one of the main problems in the study of exoplanets.In this work we aim to study how the spectral lines present in M dwarfs are independently impacted by stellar activity.We used CARMENES optical spectra of six active early- and mid-type M dwarfs to compute line-by-line RVs and study their correlation with several well-studied proxies of stellar activity.We are able to classify spectral lines based on their sensitivity to activity in five M dwarfs displaying high levels of stellar activity. We further used this line classification to compute RVs with activity-sensitive lines and less sensitive lines, enhancing or mitigating stellar activity effects in the RV time series. For specific sets of the least activity-sensitive lines, the RV scatter decreases by$\sim2$to 5 times the initial one, depending on the star. Finally, we compare these lines in the different stars analysed, finding the sensitivity to activity to vary from star to star.Despite the high density of lines and blends present in M dwarf stellar spectra, we find that a line-by-line approach is able to deliver precise RVs. Line-by-line RVs are also sensitive to stellar activity effects, and they allow for an accurate selection of activity-insensitive lines to mitigate activity effects in RV. However, we find stellar activity effects to vary in the same insensitive lines from star to star.

</div>

<div id="div_fig1">

<img src="tmp_2302.07916/./figures/J07446+035/correlation_ccfrvnzp_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png" alt="Fig7" width="100%"/>

**Figure 7. -** Line-by-line results obtained using inactive lines for J07446+035 (YZ CMi). \emph{Top left}: LAV RV time series obtained using RVs of selected inactive individual lines (colour), together with the LAV RV time series obtained using all the lines (black). The lines are selected based on the correlation of their RVs with the total spectrum RV, and their scatter. \emph{Middle left}: Number of lines used to compute the average RV in each observation, for the different selection of lines used in the \emph{top left panel}. \emph{Top right}: Distribution of the Pearson's correlation coefficient R from the correlation between the individual line RVs and the total spectrum RV. The regions in colour indicate the different selection of lines used to compute the RVs of the \emph{top left panel}. Additionally, we show in grey the distribution of R values after applying only the cut in line RV scatter (wstd). \emph{Bottom left}: Periodograms of the RV datasets plotted on the \emph{top left panel}. The horizontal lines correspond to the 0.1, 0.01, and 0.001 FAP levels, respectively. \emph{Bottom right}: Zoom in of the periodogram region around the peak corresponding to the rotation period of the star. The text indicates the period and power of the highest peak of each periodogram. (*fig:tsnhpccfrvinactivebestJ07446+035*)

</div>
<div id="div_fig2">

<img src="tmp_2302.07916/./figures/J07446+035/correlation_ccfbis_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png" alt="Fig19" width="100%"/>

**Figure 19. -** Same as Fig. \ref{fig:tsnhpccfrvinactivebestJ07446+035}, but using R values from the correlation between the individual line RVs and CCF BIS. (*fig:tsnhpccfbisinactivebestJ07446+035*)

</div>
<div id="div_fig3">

<img src="tmp_2302.07916/./figures/J07446+035/correlation_servalcrx_fitgaussianrvline/ts_nlin_histrcoeff_periodogram_all_rcoeff_cuts_-0.40_0.40_inside_stdwrvlinmax200_rcoeff_cuts_-0.30_0.30_inside_stdwrvlinmax200_rcoeff_cuts_-0.20_0.20_inside_stdwrvlinmax200_rcoeff_cuts_-0.10_0.10_inside_stdwrvlinmax200_rvline_mean_zoomprot.png" alt="Fig20" width="100%"/>

**Figure 20. -** Same as Fig. \ref{fig:tsnhpccfrvinactivebestJ07446+035}, but using R values from the correlation between the individual line RVs and CRX. (*fig:tsnhpservalcrxinactivebestJ07446+035*)

</div>